In [1]:
from torchmetrics.functional import structural_similarity_index_measure
from torchmetrics.image import PeakSignalNoiseRatio
from torchmetrics.image.lpip import LearnedPerceptualImagePatchSimilarity
import re 
import json 
import os
from PIL import Image, ImageDraw, ImageFont
from torchvision.transforms import ToTensor
import numpy as np
import matplotlib.pyplot as plt 

types = ['benchmark', 'every', 'gamma', 'noise-std', 'orientation', 'blue-noise', 'green-noise', 'red-noise', 'pos-orient', 'position', 'range', 'saturation']
types_dict = {type: {} for type in types}

psnr_metric = PeakSignalNoiseRatio(data_range=1.0)
lpips_metric = LearnedPerceptualImagePatchSimilarity()
ssim_metric = structural_similarity_index_measure

def get_timestamp(path):
    f = open(path)
    data =  json.load(f)
    ckpt_path = data['checkpoint']
    pattern = r'\d{4}-\d{2}-\d{2}_\d{6}'
    match = re.search(pattern, ckpt_path)
    timestamp = match.group()
    return timestamp

def load_image(image_path):
    image = Image.open(image_path).convert('RGB')
    return ToTensor()(image)


In [2]:
PATH_ORIGINAL_IMAGES = "C:\Stuff\Results\lego_original"
PATH_RENDERS = r"C:\Stuff\Results\renders\renders_ngp_lego_opt_off"

for folder_path in os.listdir(PATH_RENDERS)[:]:
    for filename in os.listdir(os.path.join(PATH_RENDERS, folder_path)):
        if filename.lower().endswith('img.jpg') and 'crop' not in filename:
            file_path = os.path.join(PATH_RENDERS, folder_path, filename)
            original_image = Image.open(file_path)
            original_width, original_height = original_image.size
            left = original_width / 2  # Start from the middle of the image to get the right half
            top = 0  # Start from the top edge of the image
            right = original_width  # Extend to the right edge of the image
            bottom = original_height  # Extend to the bottom edge of the image
            cropped_image = original_image.crop((left, top, right, bottom))
            new_filename = f"crop_degrated_{filename}"
            new_file_path = os.path.join(PATH_RENDERS, folder_path, new_filename)
            cropped_image.save(new_file_path)
            original_image.close()

print("Cropping completed.")


Cropping completed.


In [3]:
import os
jsons_folder = "C:\Stuff\Results\jsons\jsons_ngp_lego_opt_off"
renders_folder = r"C:\Stuff\Results\renders\renders_ngp_lego_opt_off"
# renders_folde2 = r'C:\Stuff\Results\unknown_lego_renders'

for file in os.listdir(jsons_folder):
    if file.endswith('.json'):
        json_path = os.path.join(jsons_folder, file)
        timestamp = get_timestamp(json_path)
        run_path = os.path.join(renders_folder, timestamp)
        run_path2 = os.path.join(renders_folder, timestamp)
        if os.path.isdir(run_path): #or os.path.isdir(run_path2):
            print(f"Folder with name {timestamp} exists in renders_folder., {len(os.listdir(run_path))} images found.")
        else:
            print(f"Folder with name {timestamp} does not exist in renders_folder.")


Folder with name 2024-03-19_160400 exists in renders_folder., 400 images found.
Folder with name 2024-03-04_015238 exists in renders_folder., 400 images found.
Folder with name 2024-03-04_020008 exists in renders_folder., 400 images found.
Folder with name 2024-03-04_020732 exists in renders_folder., 400 images found.
Folder with name 2024-03-04_021500 exists in renders_folder., 400 images found.
Folder with name 2024-03-04_022231 exists in renders_folder., 400 images found.
Folder with name 2024-03-04_023005 exists in renders_folder., 400 images found.
Folder with name 2024-03-04_023744 exists in renders_folder., 400 images found.
Folder with name 2024-03-04_024527 exists in renders_folder., 400 images found.
Folder with name 2024-03-04_025318 exists in renders_folder., 400 images found.
Folder with name 2024-03-04_030124 exists in renders_folder., 400 images found.
Folder with name 2024-03-04_030918 exists in renders_folder., 400 images found.
Folder with name 2024-03-04_031747 exist

In [4]:
jsons_folder = "C:\Stuff\Results\jsons\jsons_ngp_lego_opt_off"
renders_folder = r"C:\Stuff\Results\renders\renders_ngp_lego_opt_off"
PATH_ORIGINAL_IMAGES = r"C:\Stuff\Results\originals\lego_original_images"

for filename in os.listdir(jsons_folder):
    if filename.endswith(".json"):
        json_filename = filename
        json_path = os.path.join(jsons_folder, filename)
        timestamp = get_timestamp(json_path)

        run_path = os.path.join(renders_folder, timestamp)
        # print(run_path)
        if not os.path.exists(run_path):
            print(timestamp)
            os.listdir(run_path)

: 

In [6]:
psnrs, lpips_vals, ssims = [], [], []

def load_image(image_path):
    image = Image.open(image_path).convert('RGB')
    return ToTensor()(image)


all_runs_metrics = {}

jsons_folder = "C:\Stuff\Results\jsons\jsons_ngp_lego_opt_off"
renders_folder = r"C:\Stuff\Results\renders\renders_ngp_lego_opt_off"
PATH_ORIGINAL_IMAGES = r"C:\Stuff\Results\originals\lego_original_images"

for filename in os.listdir(jsons_folder):
    if filename.endswith(".json"):
        json_filename = filename
        json_path = os.path.join(jsons_folder, filename)
        timestamp = get_timestamp(json_path)

        run_path = os.path.join(renders_folder, timestamp)
        print(run_path)
        if os.path.isdir(run_path):
            psnrs, lpips_vals, ssims = [], [], []

            for filename in os.listdir(run_path):
                if filename.endswith('img.jpg') and not filename.startswith('crop_degrated_') and not filename.startswith('lego') and not filename.startswith('full_rendered'):
                    base_name = filename.split('-')[0]
                    degraded_filename = f'crop_degrated_{base_name}-img.jpg'
                    degraded_filepath = os.path.join(run_path, degraded_filename)

                    degraded_image = load_image(degraded_filepath)
                    original_image = load_image(os.path.join(PATH_ORIGINAL_IMAGES, base_name + '-img.jpg'))

                    original_image = original_image.unsqueeze(0)  # Add batch dimension
                    degraded_image = degraded_image.unsqueeze(0)  # Add batch dimension

                    psnr = psnr_metric(original_image, degraded_image)
                    ssim = ssim_metric(original_image, degraded_image)
                    lpips = lpips_metric(original_image, degraded_image)
                    psnrs.append(psnr.item())
                    ssims.append(ssim.item())
                    lpips_vals.append(lpips.item()) 
            # print(run_path)
            metrics = {
                'run': timestamp,
                'PSNR': psnrs,
                'SSIM': ssims,
                'LPIPS': lpips_vals
            }
            all_runs_metrics[json_filename[:-5]] = metrics


C:\Stuff\Results\renders\renders_ngp_lego_opt_on\2024-03-19_155548


c:\Users\danko\miniconda3\envs\nerfstudio\lib\site-packages\torchmetrics\utilities\prints.py:70: FutureWarning: Importing `spectral_angle_mapper` from `torchmetrics.functional` was deprecated and will be removed in 2.0. Import `spectral_angle_mapper` from `torchmetrics.image` instead.
  _future_warning(


C:\Stuff\Results\renders\renders_ngp_lego_opt_on\2024-03-04_095533


In [ ]:
import numpy as np
import os

# Create the 'metrics' folder if it doesn't exist
folder_path = 'metrics'
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

# Save the all_runs_metrics variable as an npy file
file_path = os.path.join(folder_path, 'ngp_lego_opt_off_metrics.npy')
np.save(file_path, all_runs_metrics)
